In [2]:
from PIL import Image
from PIL.ExifTags import TAGS
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
ih

C:\Users\Reizak\AppData\Local\Temp\ipykernel_15708\697170719.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
#Recuperation des metadata d'une image 
def getImagesInfos(imagePath):
    image = Image.open(imagePath)
    info_dict = {
    "Filename": image.filename,
    "Size": image.size,
    "Height": image.height,
    "Width": image.width,
    "Format": image.format,
    "Mode": image.mode,}
    return info_dict


In [4]:
imagesInfos = dict()
dirDsPath = "dataset_birds"

In [18]:
#Ecriture dans le fichier CSV
with open("imagesInfos.csv", 'w', newline='') as f:
    writer = csv.writer(f, delimiter=",")
    writer.writerow(["set", "birdName", "filename", "size", "height", "width", "format", "mode"])
    for setPath in os.listdir(dirDsPath):
        fullSetPath = os.path.join(dirDsPath, setPath)
        for birdPath in os.listdir(fullSetPath): 
            birdImagesList = os.listdir(os.path.join(fullSetPath, birdPath))
            for file in birdImagesList:
                infos = getImagesInfos(os.path.join(fullSetPath, birdPath, file,))
                writer.writerow([setPath, birdPath, file, infos['Size'], infos['Height'], infos['Width'], infos['Format'], infos['Mode']])

        

In [ ]:
#Informations sur la taille des images
df = pd.read_csv("imagesInfos.csv")
print("Nombre de taille d'images différentes : " + str(df['size'].nunique()))
print('\n')
print("Nombre de taille d'images différentes dans le set de test : " + str(df[df['set']== "test"]['size'].nunique()))
print("Nombre de taille d'images différentes dans le set d'entrainement : " + str(df[df['set']== "train"]['size'].nunique()))
print("Nombre de taille d'images différentes dans le set de validation : " + str(df[df['set']== "valid"]['size'].nunique()))
print('\n')

print("Nombre d'images de taille différente de 224*224 dans le set de test : " + str(df[(df['set']== "test") & (df['size'] != '(224, 224)')]['size'].nunique()))
print("Nombre d'images de taille différente de 224*224 dans le set d'entrainement : " + str(df[(df['set']== "train") & (df['size'] != '(224, 224)')]['size'].nunique()))
print("Nombre d'images de taille différente de 224*224 dans le set de validation : " + str(df[(df['set']== "valid") & (df['size'] != '(224, 224)')]['size'].nunique()))

#On voudra remplacer tout ça par des graphes


In [7]:
df['ratio_size'] = abs(df['height'] / df['width'])
df['ratio_size'].value_counts()

ratio_size
1.000000    89676
1.394737        1
0.728889        1
1.586538        1
1.941176        1
            ...  
2.155844        1
1.008696        1
0.921260        1
1.310345        1
1.666667        1
Name: count, Length: 210, dtype: int64

In [8]:
df_to_resize = df[df['size'] != '(224, 224)']
df_to_resize.groupby(['birdName', 'set']).count()

filename  size  height  width  format  mode
birdName          set                                               
LOGGERHEAD SHRIKE train         1     1       1      1       1     1
PLUSH CRESTED JAY test          5     5       5      5       5     5
                  train       200   200     200    200     200   200
                  valid         5     5       5      5       5     5

Aucune image concernant le Plush Crested Jay n'est à la bonne taille. On se propose donc de laisser tomber cette espèce.
En revanche, le Loggerhead Shrike n'a qu'une image qui pose problème. On va voir si on peut la garder en changeant sa taille. Sinon on pourra la supprimer car c'est une parmi 200

In [9]:
#Resize l'image de Loggerhead Shrike
imageToResize = df[(df['birdName']== "LOGGERHEAD SHRIKE") & (df['size'] != '(224, 224)')]
print("Il faut passer de ", imageToResize['size'].iloc[0], " à (224,224)")
img_path = os.path.join(dirDsPath, imageToResize['set'].iloc[0], imageToResize['birdName'].iloc[0], imageToResize['filename'].iloc[0])
img = Image.open(img_path)
img_resize = img.resize((224,224))
img_resize_path = os.path.join(dirDsPath+"_clean", imageToResize['set'].iloc[0], imageToResize['birdName'].iloc[0], imageToResize['filename'].iloc[0])
img_resize.save(img_resize_path)


Il faut passer de  (170, 196)  à (224,224)


In [11]:
#On va essayer de voir si la classe "PLUSH CRESTED JAY" peut être facilement conservée
df_PCJ = df_to_resize[df_to_resize['birdName'] == "PLUSH CRESTED JAY"]
df_PCJ['ratio_size'] = np.abs(df_PCJ['height'] / df_PCJ['width'])
df_PCJ['ratio_size_close_to_1'] = 1 - df_PCJ['ratio_size'] < 0.2
print(df_PCJ['ratio_size_close_to_1'].value_counts())
#Il y a trop peu d'images proches d'un carré et donc facilement resizable, on confirme la suppression de la classe "PLUSH CRESTED JAY"


ratio_size_close_to_1
True     163
False     47
Name: count, dtype: int64


C:\Users\Reizak\AppData\Local\Temp\ipykernel_10984\2806744275.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PCJ['ratio_size'] = np.abs(df_PCJ['height'] / df_PCJ['width'])
C:\Users\Reizak\AppData\Local\Temp\ipykernel_10984\2806744275.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PCJ['ratio_size_close_to_1'] = 1 - df_PCJ['ratio_size'] < 0.2


In [9]:
df['mode'].value_counts()
#Toute les images sont en couleurs

mode
RGB    89885
Name: count, dtype: int64

In [17]:
#On cherche ici la présence dans les noms de fichier de characters non numéric qui pourrait montrer des fichiers dupliqués (comme "(1)" ou encore " - copie")
df['onlyDigitFilename'] = df['filename'].apply(lambda x: x.split(".")[0].isdigit())
print(df['onlyDigitFilename'].value_counts())
print("Il n'y a que des digits, probablement pas de doublons.")

onlyDigitFilename
True    89885
Name: count, dtype: int64
Il n'y a que des digits, probablement pas de doublons.
